# Inserting to the MySQL database example

In [2]:
import pymysql.cursors
from pymysql.cursors import DictCursor
from datetime import date
from decimal import Decimal
from os import environ as ENV
from dotenv import load_dotenv
import pandas as pd
import altair as alt
import json

## Connect to the database

In [3]:
load_dotenv()

True

In [4]:
conn = pymysql.connect(host=ENV['DB_HOST'],
               user=ENV['DB_USER'],
               password=ENV['DB_PASSWORD'],
               database=ENV['DB_NAME'],
               port=int(ENV['DB_PORT']),
               cursorclass=DictCursor)

## Inserting a set of values into the transaction table

In [18]:
datetime_obj = datetime.strptime("2025-01-13 09:21:00", "%Y-%m-%d %H:%M:%S")
with conn:
    with conn.cursor() as cur:
        query = """
            INSERT INTO FACT_Transaction 
                (transaction_id,
                truck_id,
                payment_method_id,
                total,
                at)
            VALUES 
                (%s, %s, %s, %s, %s);
        """
        cur.execute(query, (2, 4, 1, 10.7, datetime_obj))
    conn.commit()

## Checking insertion and fetch

In [6]:
with conn:
    with conn.cursor() as cur:
        query = "SELECT * FROM transaction_info ORDER BY transaction_id ASC LIMIT 20;"
        cur.execute(query)
        result = cur.fetchall()
    for row in result:
        print(row)

{'transaction_id': 1, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 6)}
{'transaction_id': 2, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('8.40'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 11)}
{'transaction_id': 3, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 21)}
{'transaction_id': 4, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 36)}
{'transaction_id': 5, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 44)}
{'transaction_id': 6, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('10.60'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 59)

## Check query output for daily_report

In [5]:
query1 = """
    SELECT 
        DATE(at) date,
        SUM(total) total
    FROM transaction_info
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY;
"""

query2 = """
    SELECT
        t.truck_name,
        t.fsa_rating,
        SUM(total) total,
        COUNT(transaction_id) count,
        ROUND(AVG(total),2) avg_amount
    FROM transaction_info
    JOIN DIM_Truck t USING(truck_id)
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY
    GROUP BY truck_id
    ORDER BY truck_id ASC;
"""

query3 = """
    SELECT
        payment_method,
        SUM(total) total
    FROM transaction_info
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY
    GROUP BY payment_method;
"""

query4 = """
    SELECT
        t.truck_name,
        payment_method,
        SUM(total) total,
        ROUND(AVG(total),2) avg_amount
    FROM transaction_info
    JOIN DIM_Truck t USING(truck_id)
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY
    GROUP BY payment_method, truck_id
    ORDER BY truck_id ASC;
"""

queries = [query1, query2, query3, query4]
titles = ["total_revenue",
          "revenue_by_truck",
          "payment_method",
          "payment_method_by_truck"
          ]

In [6]:
results = {}
with conn.cursor() as cur:
    for i in range(len(queries)):
        cur.execute(queries[i])
        result = cur.fetchall()
        for row in result:
            for key, val in row.items():
                if isinstance(val, date):
                    row[key] = val.strftime('%Y-%m-%d')
                if isinstance(val, Decimal):
                    row[key] = float(val)
        results[titles[i]] = result

In [13]:
results['payment_method_by_truck']

[{'truck_name': 'Burrito Madness',
  'payment_method': 'card',
  'total': 642.8,
  'avg_amount': 7.56},
 {'truck_name': 'Burrito Madness',
  'payment_method': 'cash',
  'total': 117.3,
  'avg_amount': 7.33},
 {'truck_name': 'Kings of Kebabs',
  'payment_method': 'card',
  'total': 253.7,
  'avg_amount': 8.18},
 {'truck_name': 'Kings of Kebabs',
  'payment_method': 'cash',
  'total': 637.4,
  'avg_amount': 7.87},
 {'truck_name': 'Cupcakes by Michelle',
  'payment_method': 'card',
  'total': 502.19,
  'avg_amount': 6.2},
 {'truck_name': 'Cupcakes by Michelle',
  'payment_method': 'cash',
  'total': 47.92,
  'avg_amount': 5.99},
 {'truck_name': "Hartmann's Jellied Eels",
  'payment_method': 'card',
  'total': 34.85,
  'avg_amount': 2.32},
 {'truck_name': "Hartmann's Jellied Eels",
  'payment_method': 'cash',
  'total': 50.77,
  'avg_amount': 2.21},
 {'truck_name': 'Yoghurt Heaven',
  'payment_method': 'card',
  'total': 492.1,
  'avg_amount': 5.47},
 {'truck_name': 'Yoghurt Heaven',
  'pa

In [34]:
with open('daily_report_data.json', 'w') as f:
    json.dump(results, f, indent=4)

In [12]:
df = pd.DataFrame(results['payment_method_by_truck'])
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('truck_name:N', title='Truck Name'),
    y=alt.Y('total:Q', title='Total Amount'),
    color='payment_method:N'
).properties(
    title='Total Revenue by Truck and Payment Method',
    width=400
)
chart

alt.Chart(...)

In [7]:
df = pd.DataFrame(results['payment_method'])
chart = alt.Chart(df).mark_arc().encode(
    theta=alt.Theta('total:Q'),
    color=alt.Color('payment_method')
).properties(
    title='Payment Method Distribution',
    height=400,
    width=400
)
chart

alt.Chart(...)